In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Preprocess data
(only needs to be done once)

In [3]:
from pre import preprocess
# stores preprocessed data file
preprocessed_filename = './data/preprocessed.csv'
preprocess(i='./data/training.1600000.processed.noemoticon.csv', o=preprocessed_filename, slice=1000)

Loading ./data/training.1600000.processed.noemoticon.csv
Stripping down


100%|██████████| 1000/1000 [00:12<00:00, 78.10it/s]


Preprocessing
Done. Writing to ./data/preprocessed.csv
Success!


### Load preprocessed data

In [30]:
from parse import load_data

data = load_data(preprocessed_filename)
# TODO: train/dev/test split

### Embedding

In [50]:
from keras.preprocessing.text import Tokenizer
from emb import get_keras_embeddings_layer

MAX_WORDS = 5000
glove_filename = './data/glove.6B.50d.txt'
tokenizer = Tokenizer(lower=True, split=' ', oov_token="UNK")
emb_layer = get_keras_embeddings_layer(glove_filename, preprocessed_filename, tokenizer)

got tweets
Glove data loaded
Words not found 509


### Model

In [58]:
%load_ext autoreload
%autoreload 2
from TextClassifierModel import build_model_keras, KerasTextClassifier
import pickle


# TODO: check correct arg-assignment for tokenizer
model = build_model_keras(tokenizer, emb_layer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
building model
init
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  144400    
_________________________________________________________________
flatten_10 (Flatten)         (None, 5000)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 5000)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                320064    
_________________________________________________________________
dense_19 (Dense)             (None, 100)               6500      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 101       
Total params

### Save untrained model | load it

### Train

In [59]:
%autoreload 2
def mapping(n):
            if n == 0:
                return 0
            if n == 4:
                return 1


data_0 = data[data['target']==0]
data_4 = data[data['target']==4]

count_0, count_4 = data.target.value_counts()
print(count_0, count_4)



split = len(data['tweet'])//2

training_data = np.array(data['tweet'][:split])
training_target = np.array(data['target'][:split])
test_data = np.array(data['tweet'][split:])
test_target = np.array(data['target'][split:])

for i in range(len(training_target)):
    training_target[i] = mapping(training_target[i])
for i in range(len(test_target)):
    test_target[i] = mapping(test_target[i])
#training_target = tf.keras.utils.to_categorical(target_data, 2)
#test_target = tf.keras.utils.to_categorical(target_data, 2)

print(training_target, test_target)
#doc = np.array(data['tweet'][:1])



535 465
[0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 1
 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0
 0 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0
 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1
 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 1
 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0
 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1
 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0
 1 1 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 0 0 0 1 1 0 1 1 1 1
 0 1 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0
 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 1
 1 0 0 1 0 1 1 0 

In [60]:
%load_ext autoreload
%autoreload 2

history = model.fit(training_data, training_target)
# TODO: save trained model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[   0    0    0 ...   32    3   43]
 [   0    0    0 ...  413  919   79]
 [   0    0    0 ...  125    4  563]
 ...
 [   0    0    0 ...  100 1861  362]
 [   0    0    0 ...    4 1867 1868]
 [   0    0    0 ... 1871   16  368]]
Fit ys
[0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 1
 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0
 0 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0
 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1
 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 1
 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0
 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1
 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0
 1

### Evaluate dev

### Save model

### Evaluate production (forbidden during tuning)

### Explain prediction

In [1]:
import eli5
from eli5.lime import TextExplainer

ModuleNotFoundError: No module named 'eli5'

In [61]:



te = TextExplainer(random_state=42)
te.fit("I love candy. I like to be positive, be happy! What a lovely day", model.predict_proba)
html = te.show_prediction().data

NameError: name 'TextExplainer' is not defined